In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.layers import Conv2D, Dropout, Flatten, Dense, Input, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json, Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np

In [2]:
dataset_paths = glob.glob("dataset_characters/**/*.jpg")

In [5]:
# Arange input data and corresponding labels
X=[]
labels=[]

for image_path in dataset_paths:
  label = image_path.split(os.path.sep)[-2]
  image=load_img(image_path,target_size=(80,80))
  image=img_to_array(image)

  X.append(image)
  labels.append(label)

X = np.array(X,dtype="float16")
labels = np.array(labels)

print("[INFO] Find {:d} images with {:d} classes".format(len(X),len(set(labels))))

# perform one-hot encoding on the labels
lb = LabelEncoder()
lb.fit(labels)
labels = lb.transform(labels)
y = to_categorical(labels)

# save label file so we can use in another script
np.save('license_character_classes.npy', lb.classes_)

# split 10% of data as validation set
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.10, stratify=y, random_state=42)


# generate data augumentation method
image_gen = ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest"
                              )

[INFO] Find 37623 images with 36 classes


In [14]:
model = Sequential()
model.add(Conv2D(32, (24,24), input_shape=(80, 80, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (20,20), input_shape=(80, 80, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (20,20), input_shape=(80, 80, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(36, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00001), metrics=['accuracy'])

In [15]:
BATCH_SIZE = 64

my_checkpointer = [
                EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                ModelCheckpoint(filepath="License_character_recognition.h5", verbose=1, save_weights_only=True)
                ]

result = model.fit(image_gen.flow(trainX, trainY, batch_size=BATCH_SIZE), 
                   steps_per_epoch=len(trainX) // BATCH_SIZE, 
                   validation_data=(testX, testY), 
                   validation_steps=len(testX) // BATCH_SIZE, 
                   epochs=30, callbacks=my_checkpointer)

Epoch 1/30
529/529 [==============================] - 188s 355ms/step - loss: 4.1890 - accuracy: 0.0767 - val_loss: 2.5352 - val_accuracy: 0.3229

Epoch 00001: saving model to License_character_recognition.h5
Epoch 2/30
529/529 [==============================] - 170s 321ms/step - loss: 1.9964 - accuracy: 0.4461 - val_loss: 0.6720 - val_accuracy: 0.8225

Epoch 00002: saving model to License_character_recognition.h5
Epoch 3/30
529/529 [==============================] - 171s 324ms/step - loss: 0.9846 - accuracy: 0.7267 - val_loss: 0.4388 - val_accuracy: 0.8801

Epoch 00003: saving model to License_character_recognition.h5
Epoch 4/30
529/529 [==============================] - 171s 323ms/step - loss: 0.7073 - accuracy: 0.8075 - val_loss: 0.3571 - val_accuracy: 0.8985

Epoch 00004: saving model to License_character_recognition.h5
Epoch 5/30
529/529 [==============================] - 170s 322ms/step - loss: 0.5687 - accuracy: 0.8450 - val_loss: 0.3126 - val_accuracy: 0.9147

Epoch 00005: savi

529/529 [==============================] - 106s 200ms/step - loss: 0.1393 - accuracy: 0.9558 - val_loss: 0.1172 - val_accuracy: 0.9633- ETA: 57s - loss: 0.1378 - accuracy - ETA: 55s - loss: 0.1372 - accuracy: - ETA: 45s - loss: 0.13 - ETA: 41s - loss: 0.1372 - accuracy: 0.95 - ETA: 41s - los - ETA: 35s - loss: 0.1379 - accuracy: - ETA: 34s - loss: 0.1382 - accuracy: 0. - ETA: 33s - loss: 0.13 - ETA: 21s - loss: 0.1403 - accurac - ETA: 19s - loss: 0.1 - ETA: 15s - loss: - ETA: 1s - loss: 0.1398 - 

Epoch 00029: saving model to License_character_recognition.h5
Epoch 30/30
529/529 [==============================] - 106s 200ms/step - loss: 0.1360 - accuracy: 0.9569 - val_loss: 0.1006 - val_accuracy: 0.9700- loss: 0.1348 - accur - ETA: 49s - loss - ETA: 44s - loss: 0.1348 - accur - ETA: 42s - loss: 0.1347 - accuracy - ETA: 40s - loss: 0.1364 - accuracy: 0.95 - ETA: 40s - loss: 0.1365 - acc - ETA: 38s - loss: 0.1360  - ETA: 34s - loss: 0.1360 - accuracy: 0.9 - ETA: 34s - loss: 0.1362  - ETA:

In [16]:
model.save('plate_char_recognition.h5')